# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [6]:
import polars as pl

airlines = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv",
    null_values=["NA", ""],
)

airports = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv",
    null_values=["NA", ""],
)

flights = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv",
    null_values=["NA", ""],
    infer_schema_length=10000,
    schema_overrides={
        "arr_delay": pl.Float64,
        "dep_delay": pl.Float64,
    },
)

planes = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv",
    null_values=["NA", ""],
)

weather = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv",
    null_values=["NA", ""],
    infer_schema_length=10000,
    schema_overrides={
        "temp": pl.Float64,
        "dewp": pl.Float64,
        "humid": pl.Float64,
        "wind_speed": pl.Float64,
        "wind_gust": pl.Float64,
        "precip": pl.Float64,
        "pressure": pl.Float64,
        "visib": pl.Float64,
    },
)

ctx = pl.SQLContext()
ctx.register("airlines", airlines)
ctx.register("airports", airports)
ctx.register("flights", flights)
ctx.register("planes", planes)
ctx.register("weather", weather)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

DF ["name"]; PROJECT */1 COLUMNS


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [12]:
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("SQL Context created! Tables available:")
print(ctx.execute("SHOW TABLES"))

result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)

SQL Context created! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘
shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


/tmp/ipython-input-566239198.py:1: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


### 1.2 Find the top 10 destinations by number of flights

In [13]:
result = ctx.execute("""
SELECT dest, COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [14]:
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬──────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour            │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                  │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ str                  │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪══════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01T23:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01T12:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01T14:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01T18:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01T18:00:00Z │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …                    │
│ 2013 ┆ 9   

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [15]:
# Write your SQL query here
result = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin
""")

print(result)


shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [16]:
# First, let's check what columns are available
result = ctx.execute("""
SELECT month, COUNT(*) AS num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [17]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    flight,
    origin,
    dest,
    dep_delay,
    CASE
        WHEN dep_delay <= 15 THEN 'On-Time'
        ELSE 'Delayed'
    END AS status
FROM flights
LIMIT 20
""")

print(result)

shape: (20, 5)
┌────────┬────────┬──────┬───────────┬─────────┐
│ flight ┆ origin ┆ dest ┆ dep_delay ┆ status  │
│ ---    ┆ ---    ┆ ---  ┆ ---       ┆ ---     │
│ i64    ┆ str    ┆ str  ┆ f64       ┆ str     │
╞════════╪════════╪══════╪═══════════╪═════════╡
│ 1545   ┆ EWR    ┆ IAH  ┆ 2.0       ┆ On-Time │
│ 1714   ┆ LGA    ┆ IAH  ┆ 4.0       ┆ On-Time │
│ 1141   ┆ JFK    ┆ MIA  ┆ 2.0       ┆ On-Time │
│ 725    ┆ JFK    ┆ BQN  ┆ -1.0      ┆ On-Time │
│ 461    ┆ LGA    ┆ ATL  ┆ -6.0      ┆ On-Time │
│ …      ┆ …      ┆ …    ┆ …         ┆ …       │
│ 1806   ┆ JFK    ┆ BOS  ┆ 0.0       ┆ On-Time │
│ 1187   ┆ EWR    ┆ LAS  ┆ -1.0      ┆ On-Time │
│ 371    ┆ LGA    ┆ FLL  ┆ 0.0       ┆ On-Time │
│ 4650   ┆ LGA    ┆ ATL  ┆ 0.0       ┆ On-Time │
│ 343    ┆ EWR    ┆ PBI  ┆ 1.0       ┆ On-Time │
└────────┴────────┴──────┴───────────┴─────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [19]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier AS carrier_code,
    a.name AS airline_name,
    f.flight AS flight_number,
    f.origin,
    f.dest
FROM flights f
JOIN airlines a
  ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌──────────────┬────────────────────────┬───────────────┬────────┬──────┐
│ carrier_code ┆ airline_name           ┆ flight_number ┆ origin ┆ dest │
│ ---          ┆ ---                    ┆ ---           ┆ ---    ┆ ---  │
│ str          ┆ str                    ┆ i64           ┆ str    ┆ str  │
╞══════════════╪════════════════════════╪═══════════════╪════════╪══════╡
│ UA           ┆ United Air Lines Inc.  ┆ 1545          ┆ EWR    ┆ IAH  │
│ UA           ┆ United Air Lines Inc.  ┆ 1714          ┆ LGA    ┆ IAH  │
│ AA           ┆ American Airlines Inc. ┆ 1141          ┆ JFK    ┆ MIA  │
│ B6           ┆ JetBlue Airways        ┆ 725           ┆ JFK    ┆ BQN  │
│ DL           ┆ Delta Air Lines Inc.   ┆ 461           ┆ LGA    ┆ ATL  │
│ …            ┆ …                      ┆ …             ┆ …      ┆ …    │
│ B6           ┆ JetBlue Airways        ┆ 1806          ┆ JFK    ┆ BOS  │
│ UA           ┆ United Air Lines Inc.  ┆ 1187          ┆ EWR    ┆ LAS  │
│ B6           ┆ JetBlu

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [20]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    AVG(2013 - p.year) AS avg_age
FROM flights f
JOIN planes p
  ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier
ORDER BY avg_age DESC
""")

print(result)

shape: (16, 2)
┌─────────┬───────────┐
│ carrier ┆ avg_age   │
│ ---     ┆ ---       │
│ str     ┆ f64       │
╞═════════╪═══════════╡
│ MQ      ┆ 35.319    │
│ AA      ┆ 25.869426 │
│ DL      ┆ 16.372169 │
│ UA      ┆ 13.207691 │
│ FL      ┆ 11.385829 │
│ …       ┆ …         │
│ B6      ┆ 6.686702  │
│ F9      ┆ 4.87874   │
│ VX      ┆ 4.473643  │
│ AS      ┆ 3.33662   │
│ HA      ┆ 1.548387  │
└─────────┴───────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [21]:
# First, explore the weather table structure
result = ctx.execute("""
SELECT
    f.flight,
    f.carrier,
    f.origin,
    f.dest,
    f.dep_delay,
    w.wind_speed,
    w.precip
FROM flights f
JOIN weather w
  ON f.origin = w.origin
 AND f.time_hour = w.time_hour
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
LIMIT 20
""")

print(result)

shape: (20, 7)
┌────────┬─────────┬────────┬──────┬───────────┬────────────┬────────┐
│ flight ┆ carrier ┆ origin ┆ dest ┆ dep_delay ┆ wind_speed ┆ precip │
│ ---    ┆ ---     ┆ ---    ┆ ---  ┆ ---       ┆ ---        ┆ ---    │
│ i64    ┆ str     ┆ str    ┆ str  ┆ f64       ┆ f64        ┆ f64    │
╞════════╪═════════╪════════╪══════╪═══════════╪════════════╪════════╡
│ 21     ┆ B6      ┆ JFK    ┆ TPA  ┆ 47.0      ┆ 21.86482   ┆ 0.0    │
│ 199    ┆ B6      ┆ JFK    ┆ LAS  ┆ 116.0     ┆ 21.86482   ┆ 0.0    │
│ 4090   ┆ EV      ┆ EWR    ┆ JAX  ┆ 39.0      ┆ 20.71404   ┆ 0.0    │
│ 4231   ┆ EV      ┆ EWR    ┆ IAD  ┆ 40.0      ┆ 24.16638   ┆ 0.0    │
│ 1010   ┆ B6      ┆ JFK    ┆ BOS  ┆ 33.0      ┆ 20.71404   ┆ 0.0    │
│ …      ┆ …       ┆ …      ┆ …    ┆ …         ┆ …          ┆ …      │
│ 3368   ┆ 9E      ┆ JFK    ┆ PIT  ┆ 35.0      ┆ 20.71404   ┆ 0.0    │
│ 1465   ┆ DL      ┆ JFK    ┆ SFO  ┆ 35.0      ┆ 20.71404   ┆ 0.0    │
│ 21     ┆ AA      ┆ JFK    ┆ LAX  ┆ 41.0      ┆ 20.71404   ┆ 

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [23]:
flights_airlines_polars = (
    flights
    .join(airlines, on="carrier", how="left")
    .select(["carrier", "name", "flight", "origin", "dest"])
    .head(10)
)
print("Polars - Flights with airline names:")
print(flights_airlines_polars)
print()

result = ctx.execute("""
SELECT
    f.carrier,
    a.name,
    f.flight,
    f.origin,
    f.dest
FROM flights f
LEFT JOIN airlines a ON f.carrier = a.carrier
LIMIT 10
""")
print("SQL - Flights with airline names:")
print(result)

Polars - Flights with airline names:
shape: (10, 5)
┌─────────┬──────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ name                     ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                      ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                      ┆ i64    ┆ str    ┆ str  │
╞═════════╪══════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.    ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.    ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc.   ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways          ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.     ┆ 461    ┆ LGA    ┆ ATL  │
│ UA      ┆ United Air Lines Inc.    ┆ 1696   ┆ EWR    ┆ ORD  │
│ B6      ┆ JetBlue Airways          ┆ 507    ┆ EWR    ┆ FLL  │
│ EV      ┆ ExpressJet Airlines Inc. ┆ 5708   ┆ LGA    ┆ IAD  │
│ B6      ┆ JetBlue Airways          ┆ 79     ┆ JFK    ┆ MCO  │
│ AA      ┆ American Airlines Inc.   ┆ 301    ┆ LGA 

In [24]:
result = ctx.execute("""
SELECT
    f.flight,
    a.name AS airline_name,
    f.origin,
    ap.name AS destination_airport,
    f.dep_delay
FROM flights f
LEFT JOIN airlines a ON f.carrier = a.carrier
LEFT JOIN airports ap ON f.dest = ap.faa
WHERE f.dep_delay > 60
ORDER BY f.dep_delay DESC
LIMIT 10
""")
print("SQL - Flights with airline and destination airport info:")
print(result)

SQL - Flights with airline and destination airport info:
shape: (10, 5)
┌────────┬────────────────────────┬────────┬─────────────────────────────────┬───────────┐
│ flight ┆ airline_name           ┆ origin ┆ destination_airport             ┆ dep_delay │
│ ---    ┆ ---                    ┆ ---    ┆ ---                             ┆ ---       │
│ i64    ┆ str                    ┆ str    ┆ str                             ┆ f64       │
╞════════╪════════════════════════╪════════╪═════════════════════════════════╪═══════════╡
│ 51     ┆ Hawaiian Airlines Inc. ┆ JFK    ┆ Honolulu Intl                   ┆ 1301.0    │
│ 3535   ┆ Envoy Air              ┆ JFK    ┆ Port Columbus Intl              ┆ 1137.0    │
│ 3695   ┆ Envoy Air              ┆ EWR    ┆ Chicago Ohare Intl              ┆ 1126.0    │
│ 177    ┆ American Airlines Inc. ┆ JFK    ┆ San Francisco Intl              ┆ 1014.0    │
│ 3075   ┆ Envoy Air              ┆ JFK    ┆ Cincinnati Northern Kentucky I… ┆ 1005.0    │
│ 2391   ┆ Delta A

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [ ]:
result = ctx.execute("""
SELECT
    f.origin,
    ao.name AS origin_name,
    f.dest,
    ad.name AS dest_name,
    COUNT(*) AS total_flights,
    AVG(f.dep_delay) AS avg_dep_delay,
    100.0 * SUM(CASE WHEN f.dep_delay > 30 THEN 1 ELSE 0 END) / COUNT(*)
        AS pct_delayed_over_30
FROM flights f
JOIN airports ao ON f.origin = ao.faa
JOIN airports ad ON f.dest = ad.faa
GROUP BY f.origin, ao.name, f.dest, ad.name
ORDER BY total_flights DESC
LIMIT 10
""")

print(result)

In [ ]:
# Write your SQL query here
result = ctx.execute("""
-- Your query here
""")

# print(result)

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [25]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

result_polars = (
    flights
    .join(airlines, on="carrier", how="left")
    .join(airports, left_on="dest", right_on="faa", how="left")
    .filter(pl.col("dep_delay") > 60)
    .select([
        pl.col("flight"),
        pl.col("name").alias("airline_name"),
        pl.col("origin"),
        pl.col("name_right").alias("destination_airport"),
        pl.col("dep_delay")
    ])
    .sort("dep_delay", descending=True)
    .head(10)
)

print(result_polars)

shape: (10, 5)
┌────────┬────────────────────────┬────────┬─────────────────────────────────┬───────────┐
│ flight ┆ airline_name           ┆ origin ┆ destination_airport             ┆ dep_delay │
│ ---    ┆ ---                    ┆ ---    ┆ ---                             ┆ ---       │
│ i64    ┆ str                    ┆ str    ┆ str                             ┆ f64       │
╞════════╪════════════════════════╪════════╪═════════════════════════════════╪═══════════╡
│ 51     ┆ Hawaiian Airlines Inc. ┆ JFK    ┆ Honolulu Intl                   ┆ 1301.0    │
│ 3535   ┆ Envoy Air              ┆ JFK    ┆ Port Columbus Intl              ┆ 1137.0    │
│ 3695   ┆ Envoy Air              ┆ EWR    ┆ Chicago Ohare Intl              ┆ 1126.0    │
│ 177    ┆ American Airlines Inc. ┆ JFK    ┆ San Francisco Intl              ┆ 1014.0    │
│ 3075   ┆ Envoy Air              ┆ JFK    ┆ Cincinnati Northern Kentucky I… ┆ 1005.0    │
│ 2391   ┆ Delta Air Lines Inc.   ┆ JFK    ┆ Tampa Intl                    